In [3]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]

startTm = datetime.datetime.now()
year = "2017"
df = []
bad = []
readPath = 'F:\\SZ\\' + year + '\\***'

dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[140:]:
   
    am_snap1 = pd.read_table(data + "\\am_hq_snap_spot.txt", header=None)
    try:
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1
    except:
        assert(am_snap1.shape[1] == len(columns3))
        am_snap1.columns = columns3
    pm_snap1 = pd.read_table(data + "\\pm_hq_snap_spot.txt", header=None)
    try:
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
    except:
        assert(pm_snap1.shape[1] == len(columns3))
        pm_snap1.columns = columns3
    snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
    del am_snap1
    del pm_snap1
    snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
    snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
    print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
    print("am & pm data")
   
    

    print(datetime.datetime.now() - startTm)
    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    print(datetime.datetime.now() - startTm)
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] < 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    if len(set(gl) - set(snapshot["group"].unique())) == 0:
        s2["massive_missing"] = 0
    else:
        s2["massive_missing"] = 1
 
    # 2. check each security missing
    print(datetime.datetime.now() - startTm)
    snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
    dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
    data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "OrigTime", "time"]]
    del snapshot
    data2 = data2.sort_values(by=["StockID", "OrigTime"])
    data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
    data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
    data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
    data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
    data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
    data2 = data2.rename(columns={"75%": "75%_time"})
    vu = data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
    vu = vu.rename(columns={"volume_update":"90%"})
    data2 = pd.merge(data2, vu, on="StockID")
    data2["90%"] = data2["90%"].apply(lambda x: max(1000, x))
    data2["indicator"] = 0
    data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2 = data2[data2["indicator"] != 1]    
    if data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["90%"]) & (data2["75%_time"] <= 4)].shape[0] == 0:
        s2["security_missing"] = 0
    else:
        s2["security_missing"] = 1
        
    # 3. check close auction
    print(datetime.datetime.now() - startTm)
    dd["auction"] = np.where(dd["time"]<=145700000, 0, 1)
    dd = dd.rename(columns={"StockID": "ID"})
    s2 = pd.merge(s2, dd[["ID", "auction"]], on="ID")
    
    s2["ID"] = s2["ID"].astype(str)
    s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
    df += [s2]
    print(s2.head())
    print(data + " finished~")
    print(datetime.datetime.now() - startTm)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
df = pd.concat(df).reset_index(drop=True)
display(df)
df.to_pickle(r"F:\\SZ2017_2.pkl")
print(bad)

20170728000000000
am & pm data
0:00:28.824436
0:00:30.194279
0:00:30.502454
0:06:16.585436
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2017-07-28   10.61     10.59   10.81  10.58    10.74  81919535   
1  SZ000002  2017-07-28   23.00     23.00   23.84  23.00    23.63  41250768   
2  SZ000004  2017-07-28   22.35     22.54   22.52  22.00    22.13   1431156   
3  SZ000005  2017-07-28    4.94      4.95    4.95   4.90     4.93   3845659   
4  SZ000006  2017-07-28    8.55      8.59    8.57   8.47     8.50   9719741   

       d_amount  massive_missing  security_missing  auction  
0  8.777693e+08                0                 0        1  
1  9.719813e+08                0                 0        1  
2  3.185061e+07                0                 0        1  
3  1.893473e+07                0                 0        1  
4  8.278432e+07                0                 0        1  
F:\SZ\2017\0728 finished~
0:06:16.594412
~~~~~~~~~~~~~~~~~~~~~

0:56:36.650720
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-08-09   10.96     11.05   11.02  10.68    10.73  104232181   
1  SZ000002  2017-08-09   22.80     22.77   23.08  22.71    22.95   24498395   
2  SZ000004  2017-08-09   21.92     21.91   22.33  21.88    22.17    1067787   
3  SZ000005  2017-08-09    4.89      4.88    4.90   4.86     4.88    3325763   
4  SZ000006  2017-08-09    8.40      8.46    8.52   8.37     8.43   10341647   

       d_amount  massive_missing  security_missing  auction  
0  1.126512e+09                0                 0        1  
1  5.611456e+08                0                 0        1  
2  2.358905e+07                0                 0        1  
3  1.621937e+07                0                 0        1  
4  8.741025e+07                0                 0        1  
F:\SZ\2017\0809 finished~
0:56:36.660693
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20170810000000000
am & pm data
0:57:09.215136
0:57:10.6622

20170822000000000
am & pm data
1:48:43.765458
1:48:45.165714
1:48:45.468903
1:54:26.652795
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2017-08-22   10.48     10.46   10.66  10.37    10.65  87568181   
1  SZ000002  2017-08-22   22.78     22.62   23.40  22.65    23.21  50412349   
2  SZ000004  2017-08-22   23.29     23.31   23.44  22.90    22.93    999099   
3  SZ000005  2017-08-22    4.89      4.89    4.89   4.82     4.83   4794540   
4  SZ000006  2017-08-22    8.38      8.37    8.41   8.29     8.33   9031996   

       d_amount  massive_missing  security_missing  auction  
0  9.228819e+08                0                 0        1  
1  1.167247e+09                0                 0        1  
2  2.310021e+07                0                 0        1  
3  2.325183e+07                0                 0        1  
4  7.543357e+07                0                 0        1  
F:\SZ\2017\0822 finished~
1:54:26.661770
~~~~~~~~~~~~~~~~~~~~~

2:47:34.259660
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2017-09-01   11.28     11.28   11.39  11.15    11.21  95997689   
1  SZ000002  2017-09-01   23.26     23.23   23.37  22.74    22.88  40708029   
2  SZ000004  2017-09-01   23.16     23.15   23.26  22.96    23.08   1038181   
3  SZ000005  2017-09-01    4.89      4.89    4.89   4.85     4.87   4527190   
4  SZ000006  2017-09-01    8.49      8.45    8.50   8.40     8.42  12999103   

       d_amount  massive_missing  security_missing  auction  
0  1.080996e+09                0                 0        1  
1  9.337101e+08                0                 0        1  
2  2.394311e+07                0                 0        1  
3  2.204154e+07                0                 0        1  
4  1.097960e+08                0                 0        1  
F:\SZ\2017\0901 finished~
2:47:34.269633
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20170904000000000
am & pm data
2:48:12.904333
2:48:14.387368
2:4

20170914000000000
am & pm data
3:42:52.201714
3:42:53.810412
3:42:54.114599
3:48:58.687823
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-09-14   11.43     11.43   11.59  11.24    11.32   88308777   
1  SZ000002  2017-09-14   25.78     25.76   28.06  25.55    27.56  115069614   
2  SZ000004  2017-09-14   24.66     24.84   25.70  24.54    25.05    1399800   
3  SZ000005  2017-09-14    5.25      5.25    5.34   5.17     5.20   21031341   
4  SZ000006  2017-09-14    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  1.005776e+09                0                 0        1  
1  3.120332e+09                0                 0        1  
2  3.519613e+07                0                 0        1  
3  1.102143e+08                0                 0        1  
4  0.000000e+00                0                 0        0  
F:\SZ\2017\0914 finished~
3:48:58.697796
~~~~~~~~~~~~~~~

4:42:07.569378
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2017-09-26   11.26     11.29   11.30  10.96    11.05  96746090   
1  SZ000002  2017-09-26   26.12     26.12   27.22  26.10    26.76  59304456   
2  SZ000004  2017-09-26   23.99     24.15   24.50  23.44    24.27   1893557   
3  SZ000005  2017-09-26    5.05      5.05    5.06   5.00     5.04   5986643   
4  SZ000006  2017-09-26    0.00      9.85    0.00   0.00     0.00         0   

       d_amount  massive_missing  security_missing  auction  
0  1.074228e+09                0                 0        1  
1  1.585098e+09                0                 0        1  
2  4.566997e+07                0                 0        1  
3  3.014577e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
F:\SZ\2017\0926 finished~
4:42:07.579351
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20170927000000000
am & pm data
4:42:54.485966
4:42:56.080703
4:4

20171016000000000
am & pm data
5:35:28.298066
5:35:29.834957
5:35:30.135154
5:41:49.485866
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-10-16   11.36     11.36   11.60  11.29    11.59  103625064   
1  SZ000002  2017-10-16   26.96     26.93   26.97  26.13    26.31   31500807   
2  SZ000004  2017-10-16   25.21     25.22   25.68  24.61    25.65    1321234   
3  SZ000005  2017-10-16    5.15      5.20    5.16   5.03     5.04   12833937   
4  SZ000006  2017-10-16    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  1.190724e+09                0                 0        1  
1  8.330623e+08                0                 0        1  
2  3.314542e+07                0                 0        1  
3  6.524667e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
F:\SZ\2017\1016 finished~
5:41:49.494842
~~~~~~~~~~~~~~~

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
0,SZ000001,2017-07-28,10.61,10.59,10.81,10.58,10.74,81919535,8.777693e+08,0,0,1
1,SZ000002,2017-07-28,23.00,23.00,23.84,23.00,23.63,41250768,9.719813e+08,0,0,1
2,SZ000004,2017-07-28,22.35,22.54,22.52,22.00,22.13,1431156,3.185061e+07,0,0,1
3,SZ000005,2017-07-28,4.94,4.95,4.95,4.90,4.93,3845659,1.893473e+07,0,0,1
4,SZ000006,2017-07-28,8.55,8.59,8.57,8.47,8.50,9719741,8.278432e+07,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
119030,SZ300706,2017-10-25,66.01,60.01,66.01,66.01,66.01,191224,1.262270e+07,0,0,1
119031,SZ300707,2017-10-25,58.00,53.88,59.27,58.00,59.27,8104823,4.791648e+08,0,0,1
119032,SZ300708,2017-10-25,7.93,7.21,7.93,7.93,7.93,20740,1.644682e+05,0,0,0
119033,SZ300709,2017-10-25,81.59,74.17,81.59,81.59,81.59,43429,3.543372e+06,0,0,0


[]


In [9]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]

startTm = datetime.datetime.now()
year = "2017"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***'

dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[199:]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
        
    # am & pm
    if len(data) == 28:
        path = r'F:\SZ\2017' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()

        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1
        except:
            assert(am_snap1.shape[1] == len(columns3))
            am_snap1.columns = columns3
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1
        except:
            assert(pm_snap1.shape[1] == len(columns3))
            pm_snap1.columns = columns3
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data")
   
    # 深交所数据
    else:
        path = r'F:\SZ\2017' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\hq_snap.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\hq_snap.7z')
            bad.append(data + '\\hq_snap.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        
        snapshot = pd.read_table(path1 + "\\hq_snap.txt", header=None, encoding="UTF-8-sig")
        try:
            assert(snapshot.shape[1] == len(columns1))
            snapshot.columns = columns1
        except:
            assert(snapshot.shape[1] == len(columns3))
            snapshot.columns = columns3
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("深交所数据")
        

    print(datetime.datetime.now() - startTm)
    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    print(datetime.datetime.now() - startTm)
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] < 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    if len(set(gl) - set(snapshot["group"].unique())) == 0:
        s2["massive_missing"] = 0
    else:
        s2["massive_missing"] = 1
 
    # 2. check each security missing
    print(datetime.datetime.now() - startTm)
    snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
    dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
    data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "OrigTime", "time"]]
    del snapshot
    data2 = data2.sort_values(by=["StockID", "OrigTime"])
    data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
    data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
    data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
    data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
    data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
    data2 = data2.rename(columns={"75%": "75%_time"})
    vu = data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
    vu = vu.rename(columns={"volume_update":"90%"})
    data2 = pd.merge(data2, vu, on="StockID")
    data2["90%"] = data2["90%"].apply(lambda x: max(1000, x))
    data2["indicator"] = 0
    data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2 = data2[data2["indicator"] != 1]    
    if data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["90%"]) & (data2["75%_time"] <= 4)].shape[0] == 0:
        s2["security_missing"] = 0
    else:
        s2["security_missing"] = 1
        
    # 3. check close auction
    print(datetime.datetime.now() - startTm)
    dd["auction"] = np.where(dd["time"]<=145700000, 0, 1)
    dd = dd.rename(columns={"StockID": "ID"})
    s2 = pd.merge(s2, dd[["ID", "auction"]], on="ID")
    
    s2["ID"] = s2["ID"].astype(str)
    s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
    df += [s2]
    print(s2.head())
    print(data + " finished~")
    print(datetime.datetime.now() - startTm)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
df = pd.concat(df).reset_index(drop=True)
display(df)
df.to_pickle(r"F:\\SZ2017_3.pkl")
print(bad)

20171026000000000
am & pm data
0:01:38.402687
0:01:39.968501
0:01:40.271690
0:07:41.661832
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2017-10-26   11.25     11.27   11.32  11.12    11.18  92899607   
1  SZ000002  2017-10-26   27.30     27.43   27.88  26.91    27.53  42276685   
2  SZ000004  2017-10-26   25.05     25.05   25.05  24.80    24.87    244209   
3  SZ000005  2017-10-26    4.90      4.90    4.91   4.87     4.91   4924719   
4  SZ000006  2017-10-26    0.00      9.85    0.00   0.00     0.00         0   

       d_amount  massive_missing  security_missing  auction  
0  1.041707e+09                0                 0        1  
1  1.159749e+09                0                 0        1  
2  6.090379e+06                0                 0        1  
3  2.407637e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
J:\LEVEL2_shenzhen\2017\1026 finished~
0:07:41.672803
~~~~~~~~

20171107000000000
am & pm data
1:00:58.112335
1:00:59.655210
1:01:00.047167
1:06:44.048018
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-11-07   11.27     11.28   12.09  11.25    11.92  247716326   
1  SZ000002  2017-11-07   27.36     27.46   27.95  27.30    27.85   37981814   
2  SZ000004  2017-11-07   23.34     23.21   23.36  23.15    23.34     337426   
3  SZ000005  2017-11-07    4.56      4.56    4.65   4.52     4.63    5487604   
4  SZ000006  2017-11-07    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  2.914269e+09                0                 0        1  
1  1.052692e+09                0                 0        1  
2  7.850981e+06                0                 0        1  
3  2.511438e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
J:\LEVEL2_shenzhen\2017\1107 finished~
1:06:44.057992
~~

20171117000000000
am & pm data
2:00:54.725342
2:00:56.394879
2:00:56.759903
2:07:13.988917
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-11-17   13.17     13.10   13.46  13.03    13.18  206414972   
1  SZ000002  2017-11-17   28.85     28.94   29.60  28.60    28.90   56394399   
2  SZ000004  2017-11-17   27.66     27.81   29.38  26.10    27.88    6034363   
3  SZ000005  2017-11-17    4.63      4.64    4.63   4.46     4.59    6543906   
4  SZ000006  2017-11-17    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  2.731216e+09                0                 0        1  
1  1.639436e+09                0                 0        1  
2  1.678450e+08                0                 0        1  
3  2.973959e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
J:\LEVEL2_shenzhen\2017\1117 finished~
2:07:13.998890
~~

20171129000000000
am & pm data
2:57:09.184009
2:57:10.573295
2:57:10.926350
3:02:40.322645
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-11-29   13.73     13.70   13.93  13.47    13.82  156409493   
1  SZ000002  2017-11-29   30.78     30.75   33.83  30.28    33.82   89771259   
2  SZ000004  2017-11-29   26.47     26.37   26.59  25.06    25.52    3090467   
3  SZ000005  2017-11-29    4.23      4.23    4.28   4.19     4.24    3544301   
4  SZ000006  2017-11-29    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  2.148313e+09                0                 0        1  
1  2.894401e+09                0                 0        1  
2  7.867044e+07                0                 0        1  
3  1.498776e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
J:\LEVEL2_shenzhen\2017\1129 finished~
3:02:40.331620
~~

20171211000000000
am & pm data
3:51:53.995990
3:51:55.614662
3:51:55.924832
3:57:13.884310
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-12-11   13.08     13.09   13.57  12.88    13.50  226195273   
1  SZ000002  2017-12-11   29.82     29.82   29.95  29.30    29.93   50213546   
2  SZ000004  2017-12-11   22.91     22.91   23.09  22.61    22.61    1358985   
3  SZ000005  2017-12-11    4.10      4.11    4.17   4.09     4.17    5308204   
4  SZ000006  2017-12-11    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  2.980736e+09                0                 0        1  
1  1.487021e+09                0                 0        1  
2  3.094341e+07                0                 0        1  
3  2.197639e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
J:\LEVEL2_shenzhen\2017\1211 finished~
3:57:13.894283
~~

20171221000000000
am & pm data
4:43:37.549221
4:43:39.189834
4:43:39.493023
4:49:05.343780
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2017-12-21   13.18     13.26   13.68  13.16    13.54  148520344   
1  SZ000002  2017-12-21   28.72     28.60   30.19  28.63    29.72   59516386   
2  SZ000004  2017-12-21   22.00     22.04   22.20  21.02    22.04    1565258   
3  SZ000005  2017-12-21    4.10      4.10    4.19   4.09     4.16    4069805   
4  SZ000006  2017-12-21    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  2.005618e+09                0                 0        1  
1  1.756899e+09                0                 0        1  
2  3.394728e+07                0                 0        1  
3  1.685984e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
J:\LEVEL2_shenzhen\2017\1221 finished~
4:49:05.353754
~~

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
0,SZ000001,2017-10-26,11.25,11.27,11.32,11.12,11.18,92899607,1.041707e+09,0,0,1
1,SZ000002,2017-10-26,27.30,27.43,27.88,26.91,27.53,42276685,1.159749e+09,0,0,1
2,SZ000004,2017-10-26,25.05,25.05,25.05,24.80,24.87,244209,6.090379e+06,0,0,1
3,SZ000005,2017-10-26,4.90,4.90,4.91,4.87,4.91,4924719,2.407637e+07,0,0,1
4,SZ000006,2017-10-26,0.00,9.85,0.00,0.00,0.00,0,0.000000e+00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
96750,SZ300729,2017-12-29,34.40,35.05,34.50,31.57,32.08,6751508,2.219931e+08,0,0,1
96751,SZ300730,2017-12-29,0.00,41.55,0.00,0.00,0.00,0,0.000000e+00,0,0,0
96752,SZ300731,2017-12-29,40.93,42.51,41.63,38.26,38.26,10094061,3.973351e+08,0,0,1
96753,SZ300732,2017-12-29,86.00,88.15,86.88,80.33,80.85,5411692,4.510846e+08,0,0,1


[]


In [3]:
df = pd.concat(df).reset_index(drop=True)
display(df)
df.to_pickle(r"F:\\SZ2017_1.pkl")
print(bad)

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
0,SZ000001,2017-01-03,9.11,9.10,9.18,9.09,9.16,45984049.0,4.205952e+08,0,0,1
1,SZ000002,2017-01-03,20.55,20.55,20.88,20.55,20.73,21701669.0,4.497575e+08,0,0,1
2,SZ000004,2017-01-03,44.81,44.90,44.84,43.61,44.45,715481.0,3.173771e+07,0,0,1
3,SZ000005,2017-01-03,6.76,6.76,6.84,6.76,6.83,3704368.0,2.524036e+07,0,0,1
4,SZ000006,2017-01-03,9.50,9.45,9.78,9.47,9.67,25167430.0,2.430741e+08,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
270654,SZ300675,2017-07-27,9.34,8.49,9.34,9.34,9.34,32463.0,3.032044e+05,0,0,0
270655,SZ300676,2017-07-27,46.31,42.10,46.31,46.31,46.31,58400.0,2.704504e+06,0,0,1
270656,SZ300677,2017-07-27,42.99,39.08,42.99,42.99,42.99,107964.0,4.641372e+06,0,0,1
270657,SZ300680,2017-07-27,15.13,13.75,15.13,15.13,15.13,1949.0,2.948837e+04,0,0,0


[]
